In [ ]:
#  Download the required Dataset for analysis
# ! wget http://stat-computing.org/dataexpo/2009/2007.csv.bz2
# ! http://stat-computing.org/dataexpo/2009/2008.csv.bz2
# ! wget https://github.com/jayyanar/MachineLearning_Workbook/blob/master/2007-ord-weather-data.csv --no-check-certificate
# ! wget https://github.com/jayyanar/MachineLearning_Workbook/blob/master/2008-ord-weather-data.csv --no-check-certificate
# ! bzip2 -d 2007.csv.bz2
# ! bzip2 -d 2008.csv.bz2

In [1]:
! ls -lrt


total 4415016
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users    569231 Mar 24  2013 joda-time-2.0.jar
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users 702878193 Aug 22  2014 2007.csv
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users 113753229 Dec  9  2014 2008.csv.bz2
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users 689413344 Dec  9  2014 2008.csv
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users    169922 Jul 31 04:40 2007-ord-weather-data.csv
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users    190795 Jul 31 04:41 2008-ord-weather-data.csv
drwx------ 2 s7ed-a18f3badb92bc2-a9f6794a31ec users      4096 Aug  2 03:08 joda
-rw------- 1 s7ed-a18f3badb92bc2-a9f6794a31ec users      1553 Aug  2 04:00 preprocess1.pig


In [26]:
df = pd.read_csv("2007-ord-weather-data.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 31, saw 3


In [2]:
# For SQL-type queries (Spark)
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *

# For regression and other possible ML tools (Spark)
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics


# Important for managing features  (Spark)
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler

# For displaying and other related IPython tools...
from IPython.display import display
from IPython.html.widgets import interact

# Typycal Python tools
import sys
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import os.path

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
# ### Creating a SQL Dataframe from RDD
# 
# We now create a SQL DataFrame, this entity is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in Python, but with richer optimizations under the hood. We will utilize the recently created Spark RDD and use the Spark SQL context to create the desired data frame,

# We first create function that will allow to parse a record of our RDD into the desired format. As a reference we take a look at features_names and feature_example we just created above

def parse(r):
    try:
        x=Row(Month=int(r[1]),\
          DayofMonth=int(r[2]),\
          DayOfWeek=int(r[3]),\
          CRSDepTime=int(r[5]),\
          DepDelay=int(float(r[15])),\
          Origin=r[16],\
          Distance=int(float(r[18]))) 
    except:
        x=None  
    return x

# define hour function to obtain hour of day
def hour_ex(x): 
    h = int(str(int(x)).zfill(4)[:2])
    return h
# register as a UDF 
f = udf(hour_ex, IntegerType())


In [4]:
def prepFlightDelay(infile, orgn):
    textFile = sc.textFile(infile)
    textFileRDD = textFile.map(lambda x: x.split(','))
    header = textFileRDD.first()
    textRDD = textFileRDD.filter(lambda r: r != header)
    rowRDD = textRDD.map(lambda r: parse(r)).filter(lambda r:r != None)
    airline_df = sqlContext.createDataFrame(rowRDD)
    airline_df = airline_df.withColumn('hour', f(airline_df.CRSDepTime))
    airline_df.registerTempTable("airlineDF")
    airline_df_ORD = airline_df.filter((col("Origin") == orgn))
    airline_df_ORD_15 = airline_df_ORD.withColumn('DepDelayed', (airline_df_ORD['DepDelay']>15).cast(DoubleType()))
    return airline_df_ORD_15

In [5]:
flight_2007_ORD = prepFlightDelay('2007.csv',"ORD")
flight_2008_ORD = prepFlightDelay('2008.csv',"ORD")
flight_2007_ORD.take(5)
flight_2008_ORD.take(5)

[Row(CRSDepTime=1600, DayOfWeek=2, DayofMonth=8, DepDelay=71, Distance=719, Month=1, Origin='ORD', hour=16, DepDelayed=1.0),
 Row(CRSDepTime=1742, DayOfWeek=5, DayofMonth=25, DepDelay=31, Distance=316, Month=1, Origin='ORD', hour=17, DepDelayed=1.0),
 Row(CRSDepTime=1600, DayOfWeek=4, DayofMonth=10, DepDelay=1, Distance=719, Month=1, Origin='ORD', hour=16, DepDelayed=0.0),
 Row(CRSDepTime=650, DayOfWeek=5, DayofMonth=4, DepDelay=8, Distance=719, Month=1, Origin='ORD', hour=6, DepDelayed=0.0),
 Row(CRSDepTime=1205, DayOfWeek=4, DayofMonth=10, DepDelay=-5, Distance=925, Month=1, Origin='ORD', hour=12, DepDelayed=0.0)]

In [6]:
flight_2007_ORD.printSchema()

root
 |-- CRSDepTime: long (nullable = true)
 |-- DayOfWeek: long (nullable = true)
 |-- DayofMonth: long (nullable = true)
 |-- DepDelay: long (nullable = true)
 |-- Distance: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Origin: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- DepDelayed: double (nullable = true)



## Create an Apache® Spark machine learning model

## Prepare data

In [7]:
#split_data = flight_2007_ORD.randomSplit([0.8, 0.20], 24)
train_data = flight_2007_ORD
test_data = flight_2008_ORD


print ("Number of training records: " + str(train_data.count()))
print ("Number of testing records : " + str(test_data.count()))

Number of training records: 359169
Number of testing records : 335440


In [8]:
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline, Model

In [9]:
#In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.
stringIndexer_label = StringIndexer(inputCol="DepDelayed", outputCol="label").fit(flight_2007_ORD)
stringIndexer_org = StringIndexer(inputCol="Origin", outputCol="Orgin_Airport")

In [10]:
#In the following step, create a feature vector by combining all features together.
#[Row(CRSDepTime=1100, DayOfWeek=4, DayofMonth=25, DepDelay=-8, DepTime=1052, Dest='EWR', Distance=719, Month=1, Origin='ORD', hour=11, DepDelayed=False),
vectorAssembler_features = VectorAssembler(inputCols=["CRSDepTime","DayOfWeek","DayofMonth","DepDelay","Distance","Month","Orgin_Airport"], outputCol="features")

In [11]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features",numTrees=10)
#lr = LogisticRegressionWithSGD.train(scaledTrainData, numIterations=100)

In [15]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)
print (flight_2007_ORD)

DataFrame[CRSDepTime: bigint, DayOfWeek: bigint, DayofMonth: bigint, DepDelay: bigint, Distance: bigint, Month: bigint, Origin: string, hour: int, DepDelayed: double]


In [16]:
transform_df_pipeline = Pipeline(stages=[stringIndexer_label, stringIndexer_org, vectorAssembler_features])
transformed_df = transform_df_pipeline.fit(flight_2007_ORD).transform(flight_2007_ORD)
transformed_df.show()

+----------+---------+----------+--------+--------+-----+------+----+----------+-----+-------------+--------------------+
|CRSDepTime|DayOfWeek|DayofMonth|DepDelay|Distance|Month|Origin|hour|DepDelayed|label|Orgin_Airport|            features|
+----------+---------+----------+--------+--------+-----+------+----+----------+-----+-------------+--------------------+
|      1100|        4|        25|      -8|     719|    1|   ORD|  11|       0.0|  0.0|          0.0|[1100.0,4.0,25.0,...|
|      1500|        7|        28|      41|     925|    1|   ORD|  15|       1.0|  1.0|          0.0|[1500.0,7.0,28.0,...|
|      2000|        1|        29|      45|     316|    1|   ORD|  20|       1.0|  1.0|          0.0|[2000.0,1.0,29.0,...|
|      1900|        3|        17|      -9|     719|    1|   ORD|  19|       0.0|  0.0|          0.0|[1900.0,3.0,17.0,...|
|      1745|        5|        12|     180|     316|    1|   ORD|  17|       1.0|  1.0|          0.0|[1745.0,5.0,12.0,...|
|       930|        5|  

In [13]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [18]:
model_lr = lr.fit(transformed_df)

In [19]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(model_lr.coefficients))
print("Intercept: " + str(model_lr.intercept))

Coefficients: (7,[3],[0.007051085553957692])
Intercept: -0.995146858518747


In [20]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = model_lr.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

objectiveHistory:
0.6070093019198632
0.6030360245813051
0.5996266965654328
0.5970178159072881
0.5963395696875052
0.5956965660661498
0.595472771841593
0.5952198597394407
0.595073050772862
0.5949848693798332
0.5949714990511628
+---+--------------------+
|FPR|                 TPR|
+---+--------------------+
|0.0|                 0.0|
|0.0|4.710892525697918...|
|0.0|9.421785051395837E-5|
|0.0|1.507485608223334E-4|
|0.0|2.072792711307084...|
|0.0|2.449664113362918E-4|
|0.0| 2.92075336593271E-4|
|0.0|3.391842618502501...|
|0.0|3.862931871072293...|
|0.0|4.428238974156043...|
|0.0|4.805110376211877E-4|
|0.0|5.370417479295627E-4|
|0.0|5.935724582379377E-4|
|0.0|6.501031685463128E-4|
|0.0|7.160556639060836E-4|
|0.0|7.725863742144587E-4|
|0.0|8.291170845228337E-4|
|0.0|9.233349350367921E-4|
|0.0|0.001092927065961...|
|0.0|0.001158879561321688|
+---+--------------------+
only showing top 20 rows

areaUnderROC: 0.9999999999999999


LogisticRegression_4cd8bfd76441143fd31b

## Random Forest

In [23]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_org, vectorAssembler_features, rf, labelConverter])

In [24]:
model_rf = pipeline_rf.fit(train_data)

In [25]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy for Random Forest = %g" % accuracy)
print("Test Error for Random Forest = %g" % (1.0 - accuracy))

Accuracy for Random Forest = 0.992121
Test Error for Random Forest = 0.0078792


In [ ]:
#
# Python UDFs for our PIG script
#
from datetime import date

# this array defines the dates of holiday in 2007 and 2008
holidays = [
        date(2007, 1, 1), date(2007, 1, 15), date(2007, 2, 19), date(2007, 5, 28), date(2007, 6, 7), date(2007, 7, 4), \
        date(2007, 9, 3), date(2007, 10, 8), date(2007, 11, 11), date(2007, 11, 22), date(2007, 12, 25), \
        date(2008, 1, 1), date(2008, 1, 21), date(2008, 2, 18), date(2008, 5, 22), date(2008, 5, 26), date(2008, 7, 4), \
        date(2008, 9, 1), date(2008, 10, 13), date(2008, 11, 11), date(2008, 11, 27), date(2008, 12, 25) \
     ]
# get number of days from nearest holiday
def days_from_nearest_holiday(year, month, day):
    d = date(year, month, day)
    x = [(abs(d-h)).days for h in holidays]
    return min(x)
def to_date(year, month, day):
    td = date(year, month, day)
    return td

In [ ]:
! ls -lrta